# 6. Data Structures & Data Management — Expanded + Use Cases (EN)

This expansion adds:
- **Collections** from the standard library (`Counter`, `defaultdict`, `deque`, `heapq`)
- Robust **sorting**, **grouping**, and **joins** with lists/dicts
- Practical **data management**: CSV/TSV/JSONL, `pathlib` batch I/O, **SQLite** mini‑demo
- **Bioinformatics use cases** for genomics, proteomics, metabolomics, and microbiome

## 1) Collections you should know

- `Counter`: frequency counts and top‑k
- `defaultdict`: auto‑initialized containers (e.g., list of values per key)
- `deque`: fast append/pop on both ends
- `heapq`: priority queue / nlargest / nsmallest

In [6]:
from collections import Counter, defaultdict, deque
import heapq

# Counter
counts = Counter("MISSISSIPPI".lower())
print("counter:", counts)
print("most common 3:", counts.most_common(3))

# defaultdict of lists
groups = defaultdict(list)
for name, score in [("S1", 10), ("S2", 12), ("S1", 9)]:
    groups[name].append(score)
print("defaultdict groups:", dict(groups))

# deque as queue
q = deque()
q.append("task1"); q.appendleft("urgent"); q.append("task2")
print("deque pop left:", q.popleft(), "| remaining:", list(q))

# heapq
arr = [5,1,9,3,7,2,8]
print("nlargest 3:", heapq.nlargest(3, arr))

counter: Counter({'i': 4, 's': 4, 'p': 2, 'm': 1})
most common 3: [('i', 4), ('s', 4), ('p', 2)]
defaultdict groups: {'S1': [10, 9], 'S2': [12]}
deque pop left: urgent | remaining: ['task1', 'task2']
nlargest 3: [9, 8, 7]


## 2) Sorting tricks

- Sort by a key: `sorted(items, key=lambda x: ...)`
- Multi‑key sort with tuples: `key=lambda x: (x.city, -x.score)`
- Stable sort: sort in passes (e.g., by secondary key first)

In [7]:
records = [
    {"id":"S1", "group":"A", "score": 88},
    {"id":"S2", "group":"B", "score": 92},
    {"id":"S3", "group":"A", "score": 92},
    {"id":"S4", "group":"B", "score": 75},
]

by_group_score = sorted(records, key=lambda r: (r["group"], -r["score"], r["id"]))
by_group_score

[{'id': 'S3', 'group': 'A', 'score': 92},
 {'id': 'S1', 'group': 'A', 'score': 88},
 {'id': 'S2', 'group': 'B', 'score': 92},
 {'id': 'S4', 'group': 'B', 'score': 75}]

## 3) Grouping and "joining" in pure Python

You can **group** rows into a dict of lists, and **join** two tables by building an index dict.

In [8]:
from collections import defaultdict

# Two "tables"
samples = [
    {"sample":"S1", "group":"case"},
    {"sample":"S2", "group":"control"},
    {"sample":"S3", "group":"case"},
]
measure = [
    {"sample":"S1", "F1": 1.2, "F2": 3.4},
    {"sample":"S3", "F1": 0.7, "F2": 1.1},
    {"sample":"S2", "F1": 5.5, "F2": 2.2},
]

# Build an index by sample for fast join (hash join)
idx = {row["sample"]: row for row in measure}

joined = []
for meta in samples:
    m = idx.get(meta["sample"], {})
    joined.append({**meta, **m})  # merge dicts
joined

[{'sample': 'S1', 'group': 'case', 'F1': 1.2, 'F2': 3.4},
 {'sample': 'S2', 'group': 'control', 'F1': 5.5, 'F2': 2.2},
 {'sample': 'S3', 'group': 'case', 'F1': 0.7, 'F2': 1.1}]

## 4) Data management formats: CSV, TSV, JSONL, batch I/O

- **CSV/TSV**: rectangular, widely supported
- **JSONL** (JSON lines): one JSON object per line (easy streaming)
- Use `pathlib.Path.glob()` to batch over files

In [9]:
import csv, json, pathlib, random

# Create CSV and TSV
rows = [{"sample":f"S{i}", "reads": random.randint(800, 1500)} for i in range(1,5)]
csv_path = pathlib.Path("reads.csv")
tsv_path = pathlib.Path("reads.tsv")

with csv_path.open("w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=["sample","reads"])
    w.writeheader(); w.writerows(rows)

with tsv_path.open("w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=["sample","reads"], delimiter="\t")
    w.writeheader(); w.writerows(rows)

# JSONL
jsonl_path = pathlib.Path("reads.jsonl")
with jsonl_path.open("w", encoding="utf-8") as f:
    for r in rows:
        f.write(json.dumps(r) + "\n")

print(csv_path.read_text().strip().splitlines()[:2])
print(tsv_path.read_text().strip().splitlines()[:2])
print("JSONL first line:", jsonl_path.read_text().splitlines()[0])

['sample,reads', 'S1,1455']
['sample\treads', 'S1\t1455']
JSONL first line: {"sample": "S1", "reads": 1455}


In [ ]:
import csv, pathlib, sys, importlib

CSV_GLOB = "**/*.csv"          # recurse subfolders; use "*.csv" for current folder only
READS_KEYS = {"reads","read","read_count","readcount","num_reads","total_reads"}

def open_text_with_fallback(path: pathlib.Path):
    for enc in ("utf-8-sig", "utf-8", "latin-1", "cp1252"):
        try:
            f = path.open("r", encoding=enc, newline="")
            # force an initial decode to surface errors now
            _ = f.read(0); f.seek(0)
            return f, enc
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError("all", b"", 0, 1, "no suitable encoding")

def norm(name: str) -> str:
    return name.strip().lower().replace(" ", "").replace("-", "").replace("_", "")

def parse_int_safe(val):
    s = str(val).strip()
    if not s:
        raise ValueError("empty")
    return int(s.replace(",", "").replace(" ", ""))

def find_reads_key(fieldnames):
    if not fieldnames:
        return None
    mapping = {norm(h): h for h in fieldnames}
    for k in READS_KEYS:
        if k in mapping:
            return mapping[k]
    for k, orig in mapping.items():
        if "read" in k and "percent" not in k:
            return orig
    return None

total = 0
summary = []

for p in pathlib.Path(".").rglob("*.csv"):   # or .glob(CSV_GLOB)
    # skip macOS AppleDouble and hidden dotfiles
    if p.name.startswith("._") or p.name.startswith("."):
        continue

    try:
        f, enc = open_text_with_fallback(p)
    except UnicodeDecodeError:
        summary.append({"file": p.as_posix(), "status": "encoding_error", "enc": None, "sum": 0, "bad_rows": None})
        continue

    with f:
        try:
            r = csv.DictReader(f)
            col = find_reads_key(r.fieldnames)
            if not col:
                summary.append({"file": p.as_posix(), "status": "no_reads_col", "enc": enc, "sum": 0, "bad_rows": None})
                continue

            file_sum, bad_rows = 0, 0
            for row in r:
                try:
                    file_sum += parse_int_safe(row.get(col))
                except Exception:
                    bad_rows += 1
            total += file_sum
            summary.append({"file": p.as_posix(), "status": "ok", "enc": enc, "sum": file_sum, "bad_rows": bad_rows})
        except Exception as e:
            summary.append({"file": p.as_posix(), "status": f"parse_error:{type(e).__name__}", "enc": enc, "sum": 0, "bad_rows": None})

# print a neat report
for s in summary:
    if s["status"] == "ok":
        extra = f" (skipped {s['bad_rows']} bad rows)" if s["bad_rows"] else ""
        print(f" {s['file']} enc={s['enc']} +{s['sum']}{extra}")
    elif s["status"] == "no_reads_col":
        print(f"  {s['file']} (no 'reads' column)")
    elif s["status"] == "encoding_error":
        print(f" {s['file']} (encoding not recognized)")
    else:
        print(f" {s['file']} ({s['status']})")

print("—" * 40)
print("Total reads across CSV files:", total)

# (Optional) write a summary CSV
with open("reads_summary.csv", "w", newline="", encoding="utf-8") as out:
    w = csv.DictWriter(out, fieldnames=["file","status","enc","sum","bad_rows"])
    w.writeheader(); w.writerows(summary)
print("Wrote reads_summary.csv")

⏭️  tiny.csv (no 'reads' column)
✅ reads.csv enc=utf-8-sig +5117
————————————————————————————————————————
Total reads across CSV files: 5117
Wrote reads_summary.csv


## 5) Mini SQL database with `sqlite3` (stdlib)

- Create a table, insert rows, run queries with filtering & aggregation.
- SQLite is a file‑based database—great for small/medium datasets and teaching.

In [14]:
import sqlite3, pathlib, os

db = "lab.db"
if os.path.exists(db):
    os.remove(db)
con = sqlite3.connect(db)
cur = con.cursor()

cur.execute("CREATE TABLE sample (id TEXT PRIMARY KEY, group_name TEXT, reads INT)")
cur.executemany("INSERT INTO sample VALUES (?,?,?)", [
    ("S1","case", 1200),
    ("S2","control", 900),
    ("S3","case", 1500),
])

# Query: average reads per group
cur.execute("SELECT group_name, AVG(reads) FROM sample GROUP BY group_name")
print("avg reads:", cur.fetchall())

# Query: top sample by reads
cur.execute("SELECT id, reads FROM sample ORDER BY reads DESC LIMIT 1")
print("top sample:", cur.fetchone())

con.commit(); con.close()

avg reads: [('case', 1350.0), ('control', 900.0)]
top sample: ('S3', 1500)


## 6) Bioinformatics use cases

### (a) Genomics — FASTA index (length, GC)
Build a dictionary index from FASTA (pure-Python reader), then compute summary stats.

In [15]:
# Tiny FASTA and pure-python reader
fasta = ">seq1\nATGCGTACGT\n>seq2\nATTTGGCCAA\n"
open("tiny.fasta", "w").write(fasta)

def read_fasta(path):
    name, seq = None, []
    with open(path) as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith('>'):
                if name:
                    yield (name, ''.join(seq))
                name, seq = line[1:], []
            else:
                seq.append(line)
    if name:
        yield (name, ''.join(seq))

index = {}
for name, seq in read_fasta("tiny.fasta"):
    gc = (seq.count("G")+seq.count("C"))/len(seq)
    index[name] = {"len": len(seq), "gc": gc}

index

{'seq1': {'len': 10, 'gc': 0.5}, 'seq2': {'len': 10, 'gc': 0.4}}

### (b) Proteomics — top‑k peptides by mass using a heap

In [16]:
from decimal import Decimal
import heapq

AA_MASS = {
    "A": Decimal("71.03711"),  "C": Decimal("103.00919"),
    "D": Decimal("115.02694"), "E": Decimal("129.04259"),
    "F": Decimal("147.06841"), "G": Decimal("57.02146"),
    "H": Decimal("137.05891"), "I": Decimal("113.08406"),
    "K": Decimal("128.09496"), "L": Decimal("113.08406"),
    "M": Decimal("131.04049"), "N": Decimal("114.04293"),
    "P": Decimal("97.05276"),  "Q": Decimal("128.05858"),
    "R": Decimal("156.10111"), "S": Decimal("87.03203"),
    "T": Decimal("101.04768"), "V": Decimal("99.06841"),
    "W": Decimal("186.07931"), "Y": Decimal("163.06333"),
}
WATER = Decimal("18.01056")

def pep_mass(pep: str) -> Decimal:
    m = WATER
    for aa in pep:
        m += AA_MASS[aa]
    return m

peptides = ["ACD", "MKW", "PEPTIDE", "AAAA", "WY"]
top3 = heapq.nlargest(3, peptides, key=pep_mass)
[(p, str(pep_mass(p))) for p in top3]

[('PEPTIDE', '799.35994'), ('MKW', '463.22532'), ('WY', '367.15320')]

### (c) Metabolomics — assemble a data matrix from multiple CSV files

In [17]:
# Create 3 small CSV files with intensities for the same features
import csv, pathlib, random
features = ["F1","F2","F3","F4"]
for sid in ["S1","S2","S3"]:
    with open(f"{sid}.csv","w",newline="",encoding="utf-8") as f:
        w = csv.writer(f); w.writerow(["feature","intensity"])
        for feat in features:
            w.writerow([feat, round(random.uniform(0, 100), 2)])

# Build matrix: rows=samples, cols=features
samples = ["S1","S2","S3"]
matrix = {s:{} for s in samples}
for s in samples:
    with open(f"{s}.csv", newline="", encoding="utf-8") as f:
        r = csv.DictReader(f)
        for row in r:
            matrix[s][row["feature"]] = float(row["intensity"])

# Make a consistent table (fill missing with 0.0)
ordered = sorted(features)
table = [["sample", *ordered]]
for s in samples:
    table.append([s, *[matrix[s].get(feat, 0.0) for feat in ordered]])

for row in table:
    print(row)

['sample', 'F1', 'F2', 'F3', 'F4']
['S1', 80.77, 96.99, 43.13, 16.49]
['S2', 95.41, 22.4, 20.66, 62.79]
['S3', 57.97, 30.86, 60.35, 22.94]


### (d) Microbiome — group by taxonomic rank with `defaultdict`

In [18]:
from collections import defaultdict

# Suppose each OTU has an associated genus
otu_rows = [
    {"otu":"OTU1", "genus":"Escherichia", "count": 1200},
    {"otu":"OTU2", "genus":"Bacteroides", "count": 600},
    {"otu":"OTU3", "genus":"Escherichia", "count": 300},
]

by_genus = defaultdict(int)
for row in otu_rows:
    by_genus[row["genus"]] += row["count"]

total = sum(by_genus.values()) or 1
ra = {g: c/total for g, c in by_genus.items()}
{g: round(v,3) for g,v in ra.items()}

{'Escherichia': 0.714, 'Bacteroides': 0.286}

## 7) Exercises

1. **Top‑k GC FASTA**: Extend the FASTA index example to print the **top‑2 sequences by GC%** using `heapq.nlargest`.
2. **Join + Group**: Join `samples` and `measure` above, then compute the **mean F1** per group.
3. **SQLite filter**: In the SQLite demo, add a query that returns **all case samples with reads > 1000**.
4. **Metabolomics merge**: Extend the matrix builder to **normalize each row** so its sum is 1.0 and recompute the **max feature** per sample.
5. **Microbiome rarity**: Using `by_genus`, return a list of genera with relative abundance **< 5%**.